# Importing packages

In [1]:
using JuMP
using SCIP
using Formatting
using GLPKMathProgInterface

# Reading problem instance and defining Constants

In [2]:
instance_file = "Problema2/smt_50_250.dat"
time_limit = 60 * 60; # seconds

In [3]:
f = open(instance_file, "r")         
 
lines = readlines(f)
 
close(f)

n, m, s, t = [parse(Int, num_char) for num_char in split(lines[1])]
print("number of vertices (n): $n\n")
print("number of edges (m): $m\n")
print("initial vertice (s): $s\n")
print("final vertice (t): $t\n")

V = Array(1:n)
V_st = copy(V)
deleteat!(V_st, V_st .== s)
deleteat!(V_st, V_st .== t)
print("\nvertices (V): $V\n")
print("intermidiate vertices (V_st): $V_st\n")

D = zeros(Int64, (n, n))
G = zeros(Int64, (n, n))
P = zeros(Int64, (n, n, n))

print("\nEdges and weights: \n")
for i in 2:m+1
    u, v, d = [parse(Int, num_char) for num_char in split(lines[i])]
    print("{$u, $v} = $d\n")
    D[u, v] = d
    D[v, u] = d
    G[u, v] = 1
    G[v, u] = 1
end

for i in 1:n
    for j in 1:n
        for k in 1:n
            P[i,j,k] = abs(D[i,j] - D[j,k])
        end
    end
end

M = maximum(P);

number of vertices (n): 50
number of edges (m): 250
initial vertice (s): 1
final vertice (t): 3

vertices (V): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
intermidiate vertices (V_st): [2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

Edges and weights: 
{27, 12} = 7488
{47, 2} = 1318
{49, 25} = 4763
{44, 18} = 1006
{48, 31} = 9296
{42, 38} = 5897
{27, 3} = 8397
{32, 17} = 9265
{16, 11} = 3897
{33, 19} = 3610
{50, 24} = 3898
{47, 33} = 8085
{22, 13} = 4111
{29, 20} = 1887
{19, 7} = 727
{6, 4} = 853
{42, 14} = 5668
{30, 21} = 4313
{36, 1} = 849
{36, 4} = 9178
{47, 31} = 3099
{32, 10} = 4571
{9, 4} = 465
{40, 6} = 4712
{48, 27} = 7458
{48, 29} = 8398
{33, 20} = 2042
{43, 9} = 636
{46, 13} = 6190
{34, 28} = 4

# Defining Model

In [4]:
model = Model(SCIP.Optimizer)

set_time_limit_sec(model, time_limit);

@variable(model, C[1:n, 1:n], Bin)
@variable(model, Q[1:n, 1:n, 1:n], Bin)
@variable(model, p >= 0, Int)

@objective(model, Min, p)

# Restrições para gerar um caminho C subgrafo de G
@constraint(model, sum(C[j,s] for j = 1:n) == 0)
@constraint(model, sum(C[s,j] for j = 1:n) == 1)
@constraint(model, sum(C[j,t] for j = 1:n) == 1)
@constraint(model, sum(C[t,j] for j = 1:n) == 0)
@constraint(model, [i in V_st], sum(C[j,i] for j = 1:n) <= 1)
@constraint(model, [i in V_st], sum(C[j,i] for j = 1:n) - sum(C[i,j] for j = 1:n) == 0)
@constraint(model, [i = 1:n, j=1:n], C[i,j] - G[i,j] <= 0)

# Restrições para que  Q_{ijk} = C_{ij} ^ C_{jk}
@constraint(model, [i = 1:n, j=1:n, k=1:n], Q[i,j,k] <= (C[i,j] + C[j,k])/2)
@constraint(model, [i = 1:n, j=1:n, k=1:n], Q[i,j,k] >= C[i,j] + C[j,k] - 1)

# Restrições para que p seja o passo máximo do caminho selecionado
@constraint(model, [i = 1:n, j=1:n, k=1:n], p >= P[i,j,k] + (-M * (1 - Q[i,j,k])));

# Optimizing

In [5]:
optimize!(model)
print(objective_value(model))

presolving:
(round 1, fast)       122138 del vars, 213152 del conss, 0 add conss, 122090 chg bounds, 44158 chg sides, 44158 chg coeffs, 0 upgd conss, 0 impls, 50 clqs
(round 2, fast)       122138 del vars, 363326 del conss, 0 add conss, 122090 chg bounds, 44158 chg sides, 44158 chg coeffs, 0 upgd conss, 0 impls, 50 clqs
   (1.0s) running MILP presolver
   (1.0s) MILP presolver found nothing
(round 3, exhaustive) 122138 del vars, 363326 del conss, 0 add conss, 122091 chg bounds, 44158 chg sides, 44158 chg coeffs, 9346 upgd conss, 0 impls, 50 clqs
(round 4, exhaustive) 122163 del vars, 363326 del conss, 0 add conss, 122091 chg bounds, 44158 chg sides, 44158 chg coeffs, 9346 upgd conss, 4416 impls, 77185 clqs
(round 5, fast)       122163 del vars, 363351 del conss, 0 add conss, 122091 chg bounds, 44158 chg sides, 44158 chg coeffs, 9346 upgd conss, 4416 impls, 77185 clqs
(round 6, exhaustive) 122163 del vars, 363376 del conss, 0 add conss, 122091 chg bounds, 44158 chg sides, 44158 chg coef

501.0

In [6]:
termination_status(model)

OPTIMAL::TerminationStatusCode = 1

In [7]:
termination_status(model) == MOI.TIME_LIMIT

false

In [8]:
termination_status(model) == MOI.OPTIMAL

true

In [9]:
has_values(model)

true